In [28]:
import networkx as nx
import pandas as pd
import community
import matplotlib.pyplot as plt
import matplotlib
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import re
from collections import Counter

Importation of the needed tables :
- Senders
- Receivers
- Table that connects the Id to the name

In [2]:
senders = pd.read_csv('emails/Emails.csv', index_col='Id') # The senders of the emails
senders = senders[['SenderPersonId']]
receivers = pd.read_csv('emails/EmailReceivers.csv', index_col='Id') # The receivers of the emails
persons = pd.read_csv('emails/Persons.csv', index_col='Id') # The table that links the Ids with the names

In [3]:
senders.head()

,SenderPersonId
Id,
1,87.0
2,NaN
3,32.0
4,32.0
5,80.0


Filling of the NaN cases by 0 because there is no Id equal to 0

In [4]:
senders = senders.fillna(0)
senders['SenderPersonId'] = senders['SenderPersonId'].astype(int)

In [5]:
senders.head()

,SenderPersonId
Id,
1,87
2,0
3,32
4,32
5,80


In [6]:
receivers.head()

,EmailId,PersonId
Id,,
1,1,80
2,2,80
3,3,228
4,3,80
5,4,80


In [7]:
persons.head()

,Name
Id,
1,111th Congress
2,AGNA USEMB Kabul Afghanistan
3,AP
4,ASUNCION
5,Alec


Construction of the graph's nodes :

In [8]:
G=nx.Graph()

In [9]:
for i in range(1,persons.shape[0]+1):
    G.add_node(i)

Construction of a dictionnary that connects every email (Id of the email) to the list of the persons that are linked to it :

In [10]:
emailLinkedPersons = {}

# Adding of the sender if he's known
for i in range(1,senders.shape[0]+1):
    if senders['SenderPersonId'][i] != 0:
        emailLinkedPersons[i] = [senders['SenderPersonId'][i]]
    else:
        emailLinkedPersons[i] = []

# Adding of the receivers
for i in range(1,receivers.shape[0]+1):
    emailLinkedPersons[receivers['EmailId'][i]].append(receivers['PersonId'][i])

Construction of the graph's edges with the dictionnary :

In [11]:
for k in emailLinkedPersons.keys():
    if len(emailLinkedPersons[k])>1:
        for i in range(1,len(emailLinkedPersons[k])-1):
            for j in range(i+1,len(emailLinkedPersons[k])):
                G.add_edge(emailLinkedPersons[k][i],emailLinkedPersons[k][j])

Partition of the graph into communities by the community module :

In [12]:
partition = community.best_partition(G)

Creation of a dictionnary that connects every community id with the list of the persons that belong to it :

In [13]:
communities = {}

In [14]:
for i in partition.values():
    communities[i] = []

# Filling the communities dictionnary from the partition dictionnary
for i in partition.keys():
    communities[partition[i]].append(i)

The list of the communities

In [15]:
print('List of the communities :\n')
for i in communities.keys():
    print('Community '+str(i)+' :')
    count = 1
    for j in communities[i]:
        print(str(count)+'/ '+persons['Name'][j])
        count = count + 1
    print('\n')

List of the communities :

Community 0 :
1/ 111th Congress


Community 1 :
1/ AGNA USEMB Kabul Afghanistan


Community 2 :
1/ AP


Community 3 :
1/ ASUNCION


Community 4 :
1/ Alec


Community 5 :
1/ Alex Dupuy


Community 6 :
1/ American Beverage Association


Community 7 :
1/ Andrew Mayock


Community 8 :
1/ Andrew Shapiro
2/ Daniel Schwerin
3/ Huma Abedin
4/ Philippe Reines
5/ nuiand victoria j
6/ abedinh@stategov
7/ sullivanj@state.gov
8/ abedinh@state.gov.
9/ preines@
10/ abedinh  state ov
11/ a bedinh@state.gov
12/ preine
13/ preines sullivanjj@state.gov b6
14/ s abedinh@state.gov
15/ preines  sullivanjj@state.gov
16/ . huma abedin
17/ abedinh@stategovl
18/ preines verveerms@state.gov
19/ a bed inh@state.gov
20/ cheryl.mills sullivanjj@state.gov
21/ preines sullivanij@state.gov.
22/ abedinh@state.gove
23/ huma abed in
24/ preines sullivanjj@state.gov
25/ abedinh©state.gov
26/ preines huma abedin
27/ lew jacob i
28/ steinberg james
29/ eabedinh@state.gov
30/ abedinh@state.govr
31/

Importing the emails corpus to analyse the words used by each community

In [16]:
emails = pd.read_csv('emails/Emails.csv', index_col='Id')

In [17]:
emails.head()

,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,MetadataDocumentClass,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
Id,,,,,,,,,,,,,,,,,,,,,
1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,HRC_Email_296,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,HRC_Email_296,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,HRC_Email_296,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,HRC_Email_296,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,HRC_Email_296,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [18]:
emails = emails[['SenderPersonId','ExtractedBodyText']]

In [19]:
emails.head()

,SenderPersonId,ExtractedBodyText
Id,,
1,87.0,NaN
2,NaN,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest..."
3,32.0,Thx
4,32.0,NaN
5,80.0,"H <hrod17@clintonemail.com>\nFriday, March 11,..."


Filling the NA cases...

In [20]:
emails['SenderPersonId'] = emails['SenderPersonId'].fillna(0)
emails['SenderPersonId'] = emails['SenderPersonId'].astype(int)

In [21]:
emails['ExtractedBodyText'] = emails['ExtractedBodyText'].fillna('')

In [22]:
emails.head()

,SenderPersonId,ExtractedBodyText
Id,,
1,87,
2,0,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest..."
3,32,Thx
4,32,
5,80,"H <hrod17@clintonemail.com>\nFriday, March 11,..."


Creation of a dictionnary that connects each community with the emails that are sent by a person that belongs to it

In [23]:
emailsCommunities = {}

In [24]:
for i in communities.keys():
    emailsCommunities[i] = []

In [25]:
for i in range(1,emails.shape[0]+1):
    if emails['SenderPersonId'][i] != 0:
        emailsCommunities[partition[emails['SenderPersonId'][i]]].append(i)

In [26]:
def removeStopWords(String):
    stopwordlist = set(stopwords.words("english"))
    toRemove = ['fm','am',"n't", 'sent', 'fw', 'pm', 'fwd', 're','cc', 'attachments', "rv", "al", "aq", "la", "ben"]
    stopwordlist.update(toRemove)  
    liste=[word for word in String if word.lower() not in stopwordlist]
    return liste

def removeChar(string):
    return re.sub("[^a-zA-Z]+", " ", string)

def lowerCase(string):
    return string.lower()

Creation of a dictionnary that connects each community with the 20th most used words by its members in the emails

In [29]:
wordsCommunities = {}

In [ ]:
for i in emailsCommunities.keys():
    words = []
    for j in emailsCommunities[i]:
        cleaned_raw = emails['ExtractedBodyText'][j]
        cleaned_raw=lowerCase(cleaned_raw)
        cleaned_raw=nltk.word_tokenize(cleaned_raw)
        cleaned_raw=removeStopWords(cleaned_raw)
        cleaned_raw=' '.join(cleaned_raw)
        cleaned_raw=removeChar(cleaned_raw)
        cleaned_raw=[token for token in cleaned_raw.split() if len(token)>1]
        words = words + cleaned_raw
    wordsCommunities[i] = words

In [ ]:
for i in wordsCommunities.keys():
    wordsCommunities[i] = Counter(wordsCommunities[i]).most_common(20)

In [ ]:
for i in range(len(wordsCommunities.keys())):
    print('Community '+ str(i) + ' :')
    count_ = 0
    if len(wordsCommunities[i]) == 0:
        print('Nothing')
    for word in wordsCommunities[i]:
        count_ = count_ + 1
        print(str(count_) + '/ ' + word[0])
    print('\n')